In [8]:
workdir = '/path/to/workdir'
report_excel = 'result_to_report_demo.xlsx'

import sys
sys.path.insert(0, workdir)

In [9]:
import os
excel_path = os.path.join(workdir,report_excel)

In [35]:
import pandas as pd
import json
from collections import defaultdict

# Load the dataset from the Excel file
#df = pd.read_excel("/path/to/your/excel/file.xlsx")
df = pd.read_excel(excel_path)

# Create a structure to hold the revised JSON data
json_structure_revised = defaultdict(list)

# Iterate through the dataframe to populate the revised structure
for _, row in df.iterrows():
    condition_name = row['Condition Name']

    # Checking length of Reference Allele
    alternate_allele = row['Alternate Allele']
    breaks_after_n = 5
    if len(alternate_allele) > breaks_after_n:  # Adjust the threshold as needed
        alternate_allele = '\n'.join([alternate_allele[i:i+breaks_after_n] for i in range(0, len(alternate_allele), breaks_after_n)])


    result_details = {
        "Chromosome": row['Chromosome'],
        "Position": row['Position'],
        "Reference Allele": row['Reference Allele'],
        "Alternate Allele": alternate_allele,
        "Variant Quality": str(row['Variant Quality']).replace(" ", "\n"),
        "Filter Status": row['Filter Status'].replace(" ", "\n"),
        "ACMG Classification": row['ACMG Classification'].replace(" ","\n").replace(","," "),
        "ClinVar Significance": row['ClinVar Significance'].strip().replace(" ", "\n").replace(" Conflicting_interpretations_of_pathogenicity ","Conflicting").replace("Conflicting_interpretations_of_pathogenicity","Conflicting").replace("/","\nor\n").replace("_"," "),
        "Consequence": row['Consequence'].replace("nonsynonymous SNV", "non\nsynonymous\nSNV").replace(" ","\n"),
        "Inheritance Pattern": row['Inheritance Pattern'].replace("<br>or&nbsp;",",\n").replace("Not applicable",""), #
        "Age of Onset": row['Age of Onset'].replace("<br>", ",\n"),
        "Prevalence": row['Prevalence'].replace("1 000 000","1.000.000"),
        "gnomAD AF": row['gnomAD AF'],
        "1000g AF": row['1000g AF']
    }
    
    json_structure_revised[condition_name].append(result_details)

# Convert the defaultdict to a regular dict for JSON serialization
final_json_revised = {"condition": dict(json_structure_revised)}

# Define the output file path for the JSON data
output_file_path = os.path.join(workdir,'report_demo.json')

# Save the revised JSON data to a file
with open(output_file_path, 'w') as file:
    json.dump(final_json_revised, file, ensure_ascii=False, indent=4)
    